In [0]:
# Load in one of the tables
df2 = spark.sql("select * from default.reviews_train")
print((df2.count(), len(df2.columns)))


(3138710, 11)

In [0]:
df2.printSchema()
df2.show(5)

root
-- reviewID: integer (nullable = true)
-- overall: double (nullable = true)
-- verified: boolean (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: integer (nullable = true)
-- label: integer (nullable = true)

+--------+-------+--------+-----------+--------------+----------+---------------+--------------------+--------------------+--------------+-----+
reviewID|overall|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime|label|
+--------+-------+--------+-----------+--------------+----------+---------------+--------------------+--------------------+--------------+-----+
 1| 5.0| true| 11 5, 2015| A8LUWTIPU9CZB|0560467893| Linda Fahner|Great product, lo...| Five Stars| 1446681600| 0|
 2| 5.0| false|03 30, 2005|A1REUF3A1YCPHM|0001713353| TW Ervin II|The King, the Mic...|A story children ...| 1112140800| 0|
 3| 5.0| true|10 17, 2015|A1HP7NVNPFMA4N|0700026657| Ambrosia075|This game is a bi...|but when you do i...| 1445040000| 0|
 4| 3.0| true| 05 7, 2015|A3B6GKQQ1JJ167|0560467893|Harry Slaughter|Pretty flimsy, bu...| Meh| 1430956800| 1|
 5| 5.0| true|06 20, 2016| AVP0HXC9FG790|0001713353|Amazon Customer| The kids loved it!| Five Stars| 1466380800| 0|
+--------+-------+--------+-----------+--------------+----------+---------------+--------------------+--------------------+--------------+-----+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
df2 = df2.withColumn('date', from_unixtime('unixReviewTime').cast(DateType()))\
.withColumn("Year", year(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("Month", month(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("Quarter", quarter(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("date",to_timestamp(col("date"))).withColumn("D_O_M", date_format(col("date"), "d"))

In [0]:
#Unreadable date format
#df3 = df2.withColumn('reviewTime',to_date(df2.reviewTime, 'yyyy-MM-dd'))

In [0]:

#from pyspark.sql.functions import *
#df3 = df2.withColumn('reviewTime',to_date(df2.reviewTime, 'yyyy-MM-dd'))\
#.withColumn('day_of_week',dayofweek(df3.reviewTime))\
#.withColumn('Month',month(df4.reviewTime))\
#.withColumn('Year',year(df5.reviewTime))\
#.withColumn('Quarter',quarter(df6.reviewTime))\
#df3.show(5)

+--------+-------+--------+----------+--------------+----------+---------------+--------------------+--------------------+--------------+-----+-----------+-----+----+-------+
reviewID|overall|verified|reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime|label|day_of_week|Month|Year|Quarter|
+--------+-------+--------+----------+--------------+----------+---------------+--------------------+--------------------+--------------+-----+-----------+-----+----+-------+
 1| 5.0| true| null| A8LUWTIPU9CZB|0560467893| Linda Fahner|Great product, lo...| Five Stars| 1446681600| 0| null| null|null| null|
 2| 5.0| false| null|A1REUF3A1YCPHM|0001713353| TW Ervin II|The King, the Mic...|A story children ...| 1112140800| 0| null| null|null| null|
 3| 5.0| true| null|A1HP7NVNPFMA4N|0700026657| Ambrosia075|This game is a bi...|but when you do i...| 1445040000| 0| null| null|null| null|
 4| 3.0| true| null|A3B6GKQQ1JJ167|0560467893|Harry Slaughter|Pretty flimsy, bu...| Meh| 1430956800| 1| null| null|null| null|
 5| 5.0| true| null| AVP0HXC9FG790|0001713353|Amazon Customer| The kids loved it!| Five Stars| 1466380800| 0| null| null|null| null|
+--------+-------+--------+----------+--------------+----------+---------------+--------------------+--------------------+--------------+-----+-----------+-----+----+-------+
only showing top 5 rows

In [0]:
df2.printSchema()

root
-- reviewID: integer (nullable = true)
-- overall: double (nullable = true)
-- verified: boolean (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: integer (nullable = true)
-- label: integer (nullable = true)
-- date: timestamp (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- Quarter: integer (nullable = true)
-- D_O_M: string (nullable = true)

In [0]:
drop_list = ['reviewTime','reviewID','unixReviewTime']
df = df2.select([column for column in df2.columns if column not in drop_list])
df.show(5)

+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+
overall|verified| reviewerID| asin| reviewerName| reviewText| summary|label| date|Year|Month|Quarter|D_O_M|
+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+
 5.0| true| A8LUWTIPU9CZB|0560467893| Linda Fahner|Great product, lo...| Five Stars| 0|2015-11-05 00:00:00|2015| 11| 4| 5|
 5.0| false|A1REUF3A1YCPHM|0001713353| TW Ervin II|The King, the Mic...|A story children ...| 0|2005-03-30 00:00:00|2005| 3| 1| 30|
 5.0| true|A1HP7NVNPFMA4N|0700026657| Ambrosia075|This game is a bi...|but when you do i...| 0|2015-10-17 00:00:00|2015| 10| 4| 17|
 3.0| true|A3B6GKQQ1JJ167|0560467893|Harry Slaughter|Pretty flimsy, bu...| Meh| 1|2015-05-07 00:00:00|2015| 5| 2| 7|
 5.0| true| AVP0HXC9FG790|0001713353|Amazon Customer| The kids loved it!| Five Stars| 0|2016-06-20 00:00:00|2016| 6| 2| 20|
+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+
only showing top 5 rows

In [0]:
# Let's look at some quick summary statistics
df.describe().show()

+-------+------------------+--------------------+--------------------+------------+--------------------+--------------------+-------------------+-----------------+------------------+------------------+------------------+
summary| overall| reviewerID| asin|reviewerName| reviewText| summary| label| Year| Month| Quarter| D_O_M|
+-------+------------------+--------------------+--------------------+------------+--------------------+--------------------+-------------------+-----------------+------------------+------------------+------------------+
 count| 3138710| 3138710| 3138710| 3138480| 3138710| 3138339| 3138710| 3138710| 3138710| 3138710| 3138710|
 mean| 4.319868034957037| null| 4.429350743222517E7| NaN| 7.852610784558824E7| Infinity|0.17951674414010851|2013.635935782535| 6.350983684379888|2.4528252052594857|15.697712117398549|
 stddev|1.1246283139342776| null|2.7252114394108355E8| NaN| 7.930642215301824E8| NaN|0.38378448332434334|3.288545081656959|3.6095680596737942|1.1536578007095093| 8.82548026338377|
 min| 1.0|A0001528BGUBOEVR6T5U| 0000013765| | | | 0| 1996| 1| 1| 1|
 max| 5.0| AZZZYAYJQSDOJ| B01HIZGKOE| ~~~|~~~~~ALL GREAT ...|~~~~(,,,___,,,)(=...| 1| 2018| 12| 4| 9|
+-------+------------------+--------------------+--------------------+------------+--------------------+--------------------+-------------------+-----------------+------------------+------------------+------------------+

In [0]:
# The count of each overall rating
# 5.0 Has the most count
from pyspark.sql.functions import col
df.groupBy("overall").count().orderBy(col("overall").asc()).show()
display(df.groupBy("overall").count().orderBy("overall"))

+-------+-------+
overall| count|
+-------+-------+
 1.0| 161250|
 2.0| 137786|
 3.0| 262627|
 4.0| 551125|
 5.0|2025922|
+-------+-------+

overall,count
1.0,161250
2.0,137786
3.0,262627
4.0,551125
5.0,2025922


In [0]:
# The most common product IDs
df.groupBy("reviewerID").count().orderBy(col("count").desc()).show(10)
display(df.groupBy("reviewerID").count().orderBy(col("count").desc()).head(50))

+--------------+-----+
 reviewerID|count|
+--------------+-----+
A3V6Z4RCDGRC44| 772|
 AJKWF4W7QD4NS| 725|
A1D2C0WDCSHUWZ| 654|
A2F6N60Z96CAJI| 566|
A3W4D8XOGLWUN5| 476|
A2QHS1ZCIQOL7E| 425|
A2TCG2HV1VJP6V| 403|
A29BQ6B90Y1R5F| 384|
A1K1JW1C5CUSUZ| 359|
A2582KMXLK2P06| 322|
+--------------+-----+
only showing top 10 rows

reviewerID,count
A3V6Z4RCDGRC44,772
AJKWF4W7QD4NS,725
A1D2C0WDCSHUWZ,654
A2F6N60Z96CAJI,566
A3W4D8XOGLWUN5,476
A2QHS1ZCIQOL7E,425
A2TCG2HV1VJP6V,403
A29BQ6B90Y1R5F,384
A1K1JW1C5CUSUZ,359
A2582KMXLK2P06,322


In [0]:
# The most common product IDs
df.groupBy("asin").count().orderBy(col("count").desc()).show(10)
display(df.groupBy("asin").count().orderBy(col("count").desc()).head(50))

+----------+-----+
 asin|count|
+----------+-----+
0007420412|17660|
000711835X|16598|
0007548672|14316|
0007350899| 7155|
0007444117| 7058|
B000YGEVMI| 6621|
0007378033| 5695|
B0015TMHSI| 5294|
006017322X| 4674|
0007384289| 4574|
+----------+-----+
only showing top 10 rows

asin,count
0007420412,17660
000711835X,16598
0007548672,14316
0007350899,7155
0007444117,7058
B000YGEVMI,6621
0007378033,5695
B0015TMHSI,5294
006017322X,4674
0007384289,4574


In [0]:
#Class Imbalance
display(df.groupBy("label").count().orderBy("label"))

label,count
0,2575259
1,563451


In [0]:
#Check for Null 
df_Columns= ["overall","verified","reviewerID","asin","Year","Month","quarter","reviewerName","reviewText","summary","label"]
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df_Columns]).show()

+-------+--------+----------+----+----+-----+-------+------------+----------+-------+-----+
overall|verified|reviewerID|asin|Year|Month|quarter|reviewerName|reviewText|summary|label|
+-------+--------+----------+----+----+-----+-------+------------+----------+-------+-----+
 0| 0| 0| 0| 0| 0| 0| 230| 0| 371| 0|
+-------+--------+----------+----+----+-----+-------+------------+----------+-------+-----+

In [0]:
df.printSchema()

root
-- overall: double (nullable = true)
-- verified: boolean (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- label: integer (nullable = true)
-- date: timestamp (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- Quarter: integer (nullable = true)
-- D_O_M: string (nullable = true)

In [0]:
#df = df.na.drop(subset=["overall","verified","reviewerID","asin","Year","Month","quarter","reviewerName","reviewText","summary","label"])
df = df.na.drop()
df.show(5)
print((df.count(), len(df.columns)))

+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+
overall|verified| reviewerID| asin| reviewerName| reviewText| summary|label| date|Year|Month|Quarter|D_O_M|
+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+
 5.0| true| A8LUWTIPU9CZB|0560467893| Linda Fahner|Great product, lo...| Five Stars| 0|2015-11-05 00:00:00|2015| 11| 4| 5|
 5.0| false|A1REUF3A1YCPHM|0001713353| TW Ervin II|The King, the Mic...|A story children ...| 0|2005-03-30 00:00:00|2005| 3| 1| 30|
 5.0| true|A1HP7NVNPFMA4N|0700026657| Ambrosia075|This game is a bi...|but when you do i...| 0|2015-10-17 00:00:00|2015| 10| 4| 17|
 3.0| true|A3B6GKQQ1JJ167|0560467893|Harry Slaughter|Pretty flimsy, bu...| Meh| 1|2015-05-07 00:00:00|2015| 5| 2| 7|
 5.0| true| AVP0HXC9FG790|0001713353|Amazon Customer| The kids loved it!| Five Stars| 0|2016-06-20 00:00:00|2016| 6| 2| 20|
+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+
only showing top 5 rows

(3138109, 13)

In [0]:
from pyspark.sql.types import *
#Change Verified from Boolean to Integer
df = df.withColumn("verified", df["verified"].cast(IntegerType()))\
.withColumn("D_O_M", df["D_O_M"].cast(IntegerType()))\
.withColumn("overall", df["overall"].cast(IntegerType()))

In [0]:
df.printSchema()
df.show(5)
print((df.count(), len(df.columns)))

root
-- overall: integer (nullable = true)
-- verified: integer (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- label: integer (nullable = true)
-- date: timestamp (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- Quarter: integer (nullable = true)
-- D_O_M: integer (nullable = true)

+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+
overall|verified| reviewerID| asin| reviewerName| reviewText| summary|label| date|Year|Month|Quarter|D_O_M|
+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+
 5| 1| A8LUWTIPU9CZB|0560467893| Linda Fahner|Great product, lo...| Five Stars| 0|2015-11-05 00:00:00|2015| 11| 4| 5|
 5| 0|A1REUF3A1YCPHM|0001713353| TW Ervin II|The King, the Mic...|A story children ...| 0|2005-03-30 00:00:00|2005| 3| 1| 30|
 5| 1|A1HP7NVNPFMA4N|0700026657| Ambrosia075|This game is a bi...|but when you do i...| 0|2015-10-17 00:00:00|2015| 10| 4| 17|
 3| 1|A3B6GKQQ1JJ167|0560467893|Harry Slaughter|Pretty flimsy, bu...| Meh| 1|2015-05-07 00:00:00|2015| 5| 2| 7|
 5| 1| AVP0HXC9FG790|0001713353|Amazon Customer| The kids loved it!| Five Stars| 0|2016-06-20 00:00:00|2016| 6| 2| 20|
+-------+--------+--------------+----------+---------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+
only showing top 5 rows

(3138109, 13)

In [0]:
# set seed for reproducibility
(trainingData, testingData) = df.randomSplit([0.8, 0.2], seed = 47)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testingData.count()))

Training Dataset Count: 2511336
Test Dataset Count: 626773

In [0]:
#combine all the preprocessing steps into a pipeline.
# That way, we can run the same steps on both the training data, and testing data and beyond (new data)
# without copying and pasting any code.

from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, NGram, VectorAssembler, StringIndexer, OneHotEncoder,HashingTF, IDF, FeatureHasher 
import re
from sparknlp.annotator import ContextSpellCheckerModel
from nltk.stem import WordNetLemmatizer

# We'll tokenize the text using a simple RegexTokenizer
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")
regexTokenizer1 = RegexTokenizer(inputCol="summary", outputCol="s_words", pattern="\\W")
regexTokenizer2 = RegexTokenizer(inputCol="reviewerName", outputCol="r_words", pattern="\\W")

#n-grams
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")
ngram1 = NGram(n=2,inputCol="s_words",outputCol="s_ngrams")


# Remove standard Stopwords
stopwordsRemover = StopWordsRemover(inputCol="ngrams", outputCol="filtered")
stopwordsRemover1 = StopWordsRemover(inputCol="s_ngrams", outputCol="s_filtered")


# Vectorize the sentences using simple BOW method. Other methods are possible:
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=20)
hashingTF1 = HashingTF(inputCol="s_filtered", outputCol="s_rawFeatures", numFeatures=20)
hashingTF2 = HashingTF(inputCol="r_words", outputCol="r_rawFeatures", numFeatures=20)

#idf
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf1 = IDF(inputCol="s_rawFeatures", outputCol="s_features")
idf2 = IDF(inputCol="r_rawFeatures", outputCol="r_features")

#encode Overall
ndexer = StringIndexer(inputCols=["overall"], outputCols=["overall_ndexer"], handleInvalid="keep")
encoder = OneHotEncoder(inputCols=["overall_ndexer"],
                        outputCols=["overall_encode"])

ndexer1 = StringIndexer(inputCols=["asin"], outputCols=["asin_ndexer"], handleInvalid="keep")
encoder1 = OneHotEncoder(inputCols=["asin_ndexer"],
                        outputCols=["asin_encode"])

ndexer2 = StringIndexer(inputCols=["reviewerID"], outputCols=["reviewerID_ndexer"], handleInvalid="keep")
encoder2 = OneHotEncoder(inputCols=["reviewerID_ndexer"],
                        outputCols=["reviewerID_encode"])

ndexer3 = StringIndexer(inputCols=["verified"], outputCols=["verified_ndexer"], handleInvalid="keep")
encoder3 = OneHotEncoder(inputCols=["verified_ndexer"],
                        outputCols=["verified_encode"])


#Year Month 
hasher = FeatureHasher(inputCols=["Year", "Month", "Quarter","D_O_M"],
                       outputCol="features_date")

#Vector Assembler
vector_Assem = VectorAssembler(inputCols = ['features','s_features','r_features','overall_encode','asin_encode','reviewerID_encode','verified_encode','features_date'] , outputCol = "features3")

pipeline = Pipeline(stages=[regexTokenizer, regexTokenizer1,regexTokenizer2, ngram, ngram1, stopwordsRemover, stopwordsRemover1, hashingTF, hashingTF1, hashingTF2,idf, idf1, idf2, ndexer,encoder, ndexer1,encoder1, ndexer2,encoder2,ndexer3,encoder3,hasher, vector_Assem])


In [0]:
# Fit the pipeline to training documents.

pipelineFit = pipeline.fit(trainingData)
trainingDataTransformed = pipelineFit.transform(trainingData)
trainingDataTransformed.show(5)

+-------+--------+--------------------+----------+------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+--------------------+-----------------+--------------------+---------------+---------------+--------------------+--------------------+
overall|verified| reviewerID| asin|reviewerName| reviewText| summary|label| date|Year|Month|Quarter|D_O_M| words| s_words| r_words| ngrams| s_ngrams| filtered| s_filtered| rawFeatures| s_rawFeatures| r_rawFeatures| features| s_features| r_features|overall_ndexer|overall_encode|asin_ndexer| asin_encode|reviewerID_ndexer| reviewerID_encode|verified_ndexer|verified_encode| features_date| features3|
+-------+--------+--------------------+----------+------------+--------------------+--------------------+-----+-------------------+----+-----+-------+-----+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+--------------------+-----------------+--------------------+---------------+---------------+--------------------+--------------------+
 1| 0|A020135981U0UNEAE4JV|1890560243| mom of 4|It's a really nic...|It doesn't includ...| 1|2015-01-15 00:00:00|2015| 1| 1| 15|[it, s, a, really...|[it, doesn, t, in...| [mom, of, 4]|[it s, s a, a rea...|[it doesn, doesn ...|[it s, s a, a rea...|[it doesn, doesn ...|(20,[0,1,2,3,4,5,...|(20,[5,6,8,12,13,...|(20,[10,13,15],[1...|(20,[0,1,2,3,4,5,...|(20,[5,6,8,12,13,...|(20,[10,13,15],[2...| 3.0| (5,[3],[1.0])| 18539.0|(64420,[18539],[1...| 189397.0|(981489,[189397],...| 1.0| (2,[1],[1.0])|(262144,[38269,57...|(1308120,[0,1,2,3...|
 1| 0| A100JCBNALJFAW|B000G0LE0Y| eze543113|they keep trying ...|if it aint broke,...| 0|2007-02-08 00:00:00|2007| 2| 1| 8|[they, keep, tryi...|[if, it, aint, br...| [eze543113]|[they keep, keep ...|[if it, it aint, ...|[they keep, keep ...|[if it, it aint, ...|(20,[0,1,2,3,4,5,...|(20,[7,9,12,13,14...| (20,[19],[1.0])|(20,[0,1,2,3,4,5,...|(20,[7,9,12,13,14...|(20,[19],[2.42887...| 3.0| (5,[3],[1.0])| 10769.0|(64420,[10769],[1...| 10558.0|(981489,[10558],[...| 1.0| (2,[1],[1.0])|(262144,[38269,57...|(1308120,[0,1,2,3...|
 1| 0| A10566TEP2SO6I|B00008G97E| C. Starr|I don't reccommen...|Don't waste your ...| 1|2004-10-22 00:00:00|2004| 10| 4| 22|[i, don, t, recco...|[don, t, waste, y...| [c, starr]|[i don, don t, t ...|[don t, t waste, ...|[i don, don t, t ...|[don t, t waste, ...|(20,[0,1,2,3,4,5,...|(20,[0,1,4,9,11,1...|(20,[5,8],[1.0,1.0])|(20,[0,1,2,3,4,5,...|(20,[0,1,4,9,11,1...|(20,[5,8],[2.4393...| 3.0| (5,[3],[1.0])| 52650.0|(64420,[52650],[1...| 30718.0|(981489,[30718],[...| 1.0| (2,[1],[1.0])|(262144,[38269,57...|(1308120,[0,1,2,3...|
 1| 0| A105GWGM7PDAI2|B00004TJCL| jluszcz|Counter-Strike is...|Great Game, But I...| 1|2002-08-19 00:00:00|2002| 8| 3| 19|[counter, strike,...|[great, game, but...| [jluszcz]|[counter strike, ...|[great game, game...|[counter strike, ...|[great game, game...|(20,[0,1,2,3,4,5,...|(20,[0,3,4,7,10,1...| (20,[10],[1.0])|(20,[0,1,2,3,4,5,...|(20,[0,3,4,7,10,1...|(20,[10],[2.10931...| 3.0| (5,[3],[1.0])| 13817.0|(64420,[13817],[1...| 30720.0|(981489,[30720],[...| 1.0| (2,[1],[1.0])|(262144,[38269,57...|(1308120,[0,1,2,3...|
 1| 0| A109CREBBPLP9L|B000B69E96| The Dog Zoo|I bought this gam...| Not really a game.| 0|2014-03-29 00:00:00|2014| 3| 1| 29|[i, bought, this,...|[not, really, a, ...|[the, dog, zoo]|[i bought, bought...|[not really, real...|[i bought, bought...|[not really, real...|(20,[0,1,2,3

In [0]:
from pyspark.ml.classification import LogisticRegression

# More classification docs: https://spark.apache.org/docs/latest/ml-classification-regression.html

lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0, featuresCol='features3', labelCol='label')
lrModel = lr.fit(trainingDataTransformed)

In [0]:
# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary

print("Training Accuracy:  " + str(trainingSummary.accuracy))
print("Training Precision: " + str(trainingSummary.precisionByLabel))
print("Training Recall:    " + str(trainingSummary.recallByLabel))
print("Training FMeasure:  " + str(trainingSummary.fMeasureByLabel()))
print("Training AUC:       " + str(trainingSummary.areaUnderROC))

Training Accuracy: 0.8847481977720225
Training Precision: [0.8852110237645331, 0.8797399834312396]
Training Recall: [0.9876008832593239, 0.41460693100495577]
Training FMeasure: [0.9336070383384281, 0.5635989711022052]
Training AUC: 0.9614496152691401

In [0]:
trainingSummary.roc.show()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 warnings.warn(
+--------------------+--------------------+
 FPR| TPR|
+--------------------+--------------------+
 0.0| 0.0|
1.436831204694511...|0.004242996885312839|
2.938384535726613...| 0.00839722814540993|
4.746289264756644...| 0.01246269378029127|
6.739730755954436E-4| 0.01638810698427665|
8.797894373489818E-4| 0.0202563156742341|
0.001109337245426...| 0.02405745700291743|
0.001342336900241...|0.027829009789862177|
0.001567138419054194| 0.03159859000735769|
0.001797980669658...| 0.03539578619714254|
0.002043493268898861| 0.03907265565052381|
0.002287711425612...| 0.04275938795115129|
0.002541853641698309| 0.04639286087664932|
0.002811097687262...|0.049988854982611804|
0.003060493614083...|0.053675587283239276|
0.003334052468070...|0.057257773403057086|
0.003626596479116327| 0.0607788098699485|
0.003912668277528478| 0.06431168175353534|
0.004203054884363134| 0.06780312967868816|
0.004493441491197791| 0.0713103581594349|
+--------------------+--------------------+
only showing top 20 rows

In [0]:
# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
for objective in objectiveHistory:
    print(objective)

0.4706052680711008
0.43468140668242083
0.4118057989487282
0.40311256099631737
0.39791443790987524
0.3964509744366182
0.39308536437977154
0.39223294796706504
0.39188343362926925
0.3917508106999225
0.39153009418789025
0.39137065268724974
0.3913217184222271
0.39129362743946866
0.39129054951089925
0.3912888692710857
0.39128866979801746
0.391288515427983
0.3912884881196431
0.3912884799877096
0.39128846929584077
0.391288468203598
0.3912884669603933
0.39128846674835716
0.39128846672474493
0.391288466704653
0.3912884667024395
0.3912884667002345
0.3912884667001536
0.3912884667000015
0.39128846669996137

In [0]:
testingDataTransform = pipelineFit.transform(testingData)
testingDataTransform.show(5)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictions = lrModel.transform(testingDataTransform)
#predictions.show(5)

evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
#print('Test Area Under ROC', evaluator.evaluate(predictions))

In [0]:
# Load in the tables
test_df = spark.sql("select * from default.reviews_test")
test_df.show(5)
print((test_df.count(), len(test_df.columns)))

+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+
reviewID|overall|verified| reviewTime| reviewerID| asin|reviewerName| reviewText| summary|unixReviewTime|
+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+
80000001| 4.0| false|07 27, 2015|A1JGAP0185YJI6|0700026657| travis|I played it a whi...|But in spite of t...| 1437955200|
80000002| 5.0| true| 03 3, 2014|A1WK5I4874S3O2|0700026657| WhiteSkull|I bought this gam...|A very good game ...| 1393804800|
80000003| 5.0| true|01 12, 2013|A1YDQQJDRHM0FJ|0001713353| Leila|I am very happy w...|One of our famili...| 1357948800|
80000004| 5.0| true|11 20, 2011|A2E6AHFDJ3JBAZ|0681795107| robosolo|I purchased two o...|Insulated stainle...| 1321747200|
80000005| 5.0| false|06 28, 2011|A38NXTZUFB1O2K|0700099867| FiSH|I'm not quite fin...| Best in the series!| 1309219200|
+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+
only showing top 5 rows

(348621, 10)

In [0]:
test_df = test_df.fillna(value='abc')

In [0]:

test_df = test_df.withColumn('date', from_unixtime('unixReviewTime').cast(DateType()))\
.withColumn("Year", year(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("Month", month(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("Quarter", quarter(from_unixtime('unixReviewTime').cast(DateType())))\
.withColumn("date",to_timestamp(col("date"))).withColumn("D_O_M", date_format(col("date"), "d"))

In [0]:
test_df = test_df.withColumn("verified", test_df["verified"].cast(IntegerType()))\
.withColumn("D_O_M", test_df["D_O_M"].cast(IntegerType()))\
.withColumn("overall", test_df["overall"].cast(IntegerType()))

In [0]:
drop_list = ['date']
test_df = test_df.select([column for column in test_df.columns if column not in drop_list])
test_df.show(5)
print((test_df.count(), len(test_df.columns)))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2108068629730064> in <module> 
 1 drop_list = [ 'date' ] 
 ----> 2 test_df = test_df . select ( [ column for column in test_df3 . columns if column not in drop_list ] ) 
 3 #test_df.show(5) 
 4 #print((test_df.count(), len(test_df.columns))) 

 NameError : name 'test_df3' is not defined

In [0]:
test_df.printSchema()

root
-- reviewID: integer (nullable = true)
-- overall: double (nullable = true)
-- verified: boolean (nullable = true)
-- asin: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)

In [0]:
test_df_Transform = pipelineFit.transform(test_df)
test_df_Transform.show(5)

+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+-------------------+----+-----+-------+-----+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+--------------------+-----------------+--------------------+---------------+---------------+--------------------+--------------------+
reviewID|overall|verified| reviewTime| reviewerID| asin|reviewerName| reviewText| summary|unixReviewTime| date|Year|Month|Quarter|D_O_M| words| s_words| r_words| ngrams| s_ngrams| filtered| s_filtered| rawFeatures| s_rawFeatures| r_rawFeatures| features| s_features| r_features|overall_ndexer|overall_encode|asin_ndexer| asin_encode|reviewerID_ndexer| reviewerID_encode|verified_ndexer|verified_encode| features_date| features3|
+--------+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+-------------------+----+-----+-------+-----+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+--------------+-----------+--------------------+-----------------+--------------------+---------------+---------------+--------------------+--------------------+
80000001| 4| 0|07 27, 2015|A1JGAP0185YJI6|0700026657| travis|I played it a whi...|But in spite of t...| 1437955200|2015-07-27 00:00:00|2015| 7| 3| 27|[i, played, it, a...|[but, in, spite, ...| [travis]|[i played, played...|[but in, in spite...|[i played, played...|[but in, in spite...|(20,[0,1,2,3,4,5,...|(20,[3,4,6,7,10,1...|(20,[12],[1.0])|(20,[0,1,2,3,4,5,...|(20,[3,4,6,7,10,1...|(20,[12],[2.17300...| 1.0| (5,[1],[1.0])| 35312.0|(64420,[35312],[1...| 10872.0|(981489,[10872],[...| 1.0| (2,[1],[1.0])|(262144,[38269,57...|(1308120,[0,1,2,3...|
80000002| 5| 1| 03 3, 2014|A1WK5I4874S3O2|0700026657| WhiteSkull|I bought this gam...|A very good game ...| 1393804800|2014-03-03 00:00:00|2014| 3| 1| 3|[i, bought, this,...|[a, very, good, g...|[whiteskull]|[i bought, bought...|[a very, very goo...|[i bought, bought...|[a very, very goo...|(20,[0,1,2,3,4,5,...|(20,[0,2,6,14,15,...| (20,[8],[1.0])|(20,[0,1,2,3,4,5,...|(20,[0,2,6,14,15,...|(20,[8],[2.573014...| 0.0| (5,[0],[1.0])| 35312.0|(64420,[35312],[1...| 88697.0|(981489,[88697],[...| 0.0| (2,[0],[1.0])|(262144,[38269,57...|(1308120,[0,1,2,3...|
80000003| 5| 1|01 12, 2013|A1YDQQJDRHM0FJ|0001713353| Leila|I am very happy w...|One of our famili...| 1357948800|2013-01-12 00:00:00|2013| 1| 1| 12|[i, am, very, hap...|[one, of, our, fa...| [leila]|[i am, am very, v...|[one of, of our, ...|[i am, am very, v...|[one of, of our, ...|(20,[0,1,3,4,6,7,...|(20,[7,8,10,13,16...|(20,[17],[1.0])|(20,[0,1,3,4,6,7,...|(20,[7,8,10,13,16...|(20,[17],[2.38816...| 0.0| (5,[0],[1.0])| 29452.0|(64420,[29452],[1...| 981489.0| (981489,[],[])| 0.0| (2,[0],[1.0])|(262144,[38269,57...|(1308120,[0,1,3,4...|
80000004| 5| 1|11 20, 2011|A2E6AHFDJ3JBAZ|0681795107| robosolo|I purchased two o...|Insulated stainle...| 1321747200|2011-11-20 00:00:00|2011| 11| 4| 20|[i, purchased, tw...|[insulated, stain...| [robosolo]|[i purchased, pur...|[insulated stainl...|[i purchased, pur...|[insulated stainl...|(20,[0,1,2,3,4,5,...|(20,[1,7,10,12,17...| (20,[5],[1.0])|(20,[0,1,2,3,4,5,...|(20,[1,7,10,12,17...|(20,[5],[2.439375...| 0.0| (5,[0],[1.0])| 40485.0|(64420,[40485],[1...| 387961.0|(981489,[387961],...| 0.0| (2,[0],[1.0])|(262144,[38269,57...|(1308120,[0,1,2,3...|
80000005| 5| 0|06 28, 2011|A38NXTZUFB1O2K|0700099867| FiSH|I'm not quite fin...| Best in the series!| 1309219200|2011-06-28 00:00:0

In [0]:
predictions = lrModel.transform(test_df_Transform)

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

probelement=udf(lambda v:float(v[1]),FloatType())
submission_data = predictions.select('reviewID', probelement('probability')).withColumnRenamed('<lambda>(probability)', 'label')

In [0]:
display(submission_data.select('reviewID', 'label'))

reviewID,label
80000001,0.08320157
80000002,0.1034075
80000003,0.11689244
80000004,0.568138
80000005,0.5428738
80000006,0.15031734
80000007,0.10885752
80000008,0.6264012
80000009,0.14836189
80000010,0.5272095
